# Temporal Difference Learning

- <font color=blue>和MC learning一样，TD learning也是model-free类的方法。但是TD相比MC而言的优势体现在它的算法是incremental形式。</font>

#### 从RM到TD发生了哪些本质变化

- <font color=norange>RM算法是一种通过迭代法求解函数期望值的方法，但不能直接用于强化学习，TD算法是RM算法在强化学习的条件下进行改造后得到的算法。 </font>

1. RM算法要点：
   - RM的迭代式中涉及单一变量x，估计所使用的样本是被估计目标Ef(x)的真实值的抽样，比如均值问题中 $\hat g(w,\eta)是g(w)=w-Ef(x)的样本$
   - 所以<font color=blue>**RM算法本质上是supervised learning算法**</font>。迭代$w_{t+1}=w_t-\alpha_t(w_t-f(x_t))$过程中，估计量w通过逼近Ef(x)的真实样本f(x)而向Ef(x)收敛。

2. TD算法要点：
   - 但强化学习中，涉及的变量是多变量 $s_i\in S$ 。更重要的是，此时学习过程中没有被估计目标真实值的样本。
     - 如: 估计$v(s)$时，$v_t(s)的样本只有r_{t+1}+\gamma v_t(s')，而不是r_{t+1}+\gamma v^{\pi}(s')$
   - TD算法在RM算法基础上放宽了抽样条件，既然没有真实值，就<font color=green>**用当前对真实值的最佳估计来代替真实值**</font>——这就是TD算法相对RM算法做出的最大变化。
   - 好在，<font color=blue>**在满足一定条件的时候，TD算法仍然满足almost surely convergence。**</font>
     - RM的收敛性质是用Dvorotzky's theorem证明的，而本节介绍的TD类算法中的tabular Sarsa(on-policy)和tabular Q-learning(off-policy)的收敛性质是用Dvorotzky's theorem的多变量扩展形式证明的。
     - <font color=red>而下一章中Sarsa/Q-learning with nonlinear function approximation就无法保证almost surely convergence</font>

## I. reinforcement learning的基本概念

1. on-policy和off-policy
   -  **behavior policy**：用于生成经验样本的策略
   -  **target policy**: 学习的目标，在学习过程中不断被更新以converge到最优策略
   -  <font color=blue>**on-policy learning**</font>：behavior policy和target policy是同一个策略
   -  <font color=blue>**off-policy learning**</font>：behavior policy和target policy不是同一策略
- <font color=green>**off-policy的优点**</font>：用别的策略生成的样本来learn最优策略会带来一些好处。比如：
  1. 这个被学习的策略可以是人类的操作。
  2. <font color=red>hebavior policy可以故意设为有很强的exploration能力的policy，这会提升模型learn最优策略的能力。</font>因为，估计action value要对所有state-action pair估计$Q(s,a)$。这意味着每个pair都需要大量sample。如果target policy本身没有很强的exploration的能力，就很难做到。比如Sarsa用$\epsilon -greedy$来保证一定的exploration ability，但$\epsilon$的值通常设置得很小，exploration的效果有限。但是如果能用一个有很强的exploration能力的策略专门来generate episodes，然后用off-policy learning来学习最优策略，那么learning效率会有大幅提升。

2. online planning和offline planning
   - ① **offline planning**：the agent updates the values and policies using pre-collected experience data without interacting with the environment.
   - ② **online planning**：the agent updates the values and policies while interacting with the environment. 

3. sample and episode
   - ① **sample**: 在online planning中，agent的一次行为反馈过程$(s, a, s^{'}, r)$称为一个sample。
   - ② **episode**: 在online planning中，agent持续take action，并collect samples，直到达到terminal state的整个过程称为一个episode。通常要经历多个episods才能完成exploration。

4. model-based learning and model-free learning 
   - ① <font color=blue>**model-based learning**</font>：先利用exploration得到的episodes数据，来构造model估计transition function和rewards function，然后就可以将问题转化为MDP问题求解。
     - a. 估计transition function：$\hat T(s, a, s^{'})=\frac{\#(s, a, s^{'})}{\#(s, a)}$ 
     - b. 估计reward function：$\hat R(s, a, s^{'})= R(s, a, s^{'})$ 
   - ② <font color=blue>**model-free learning**</font>：不估计transition function和rewards function，而是直接估计value或者Q-values of states.比如MC和TD。

## II.basic TD：TD learning of state value

### II.1 思路：用RM迭代估计$V^{\pi}(s)$

- <font color=orange>和MC based direct learning相似，用抽样的方法来估$V^{\pi}(s)$ 。但改用RM算法的迭代方式而不是MC方法来估。同样，估计了$V^{\pi}(s)$之后，他们不能直接做policy improvement，还需要reward function和transition function的信息才能将$V^{\pi}(s)$ 转化为$Q^{\pi}(s,a)$才能做policy improvement。所以这个方法不会在实践中使用，只是作为引例方便介绍后续相关算法。</font>

### II.2 basic TD迭代式

#### II.2.1 迭代式

$$\begin{align} 
iterate: \\
& v^{\pi }_{t+1}(s_t) \leftarrow v_t^{\pi}(s_t)-\alpha_t \left [v_t^{\pi}(s_t)-\left ( r_{t+1} + \gamma v^{\pi}_t(s_{t+1}) \right ) \right ]\\
& v_{t+1}^{\pi}(s) \leftarrow v^{\pi}_t(s), \forall s\ne s_t
\end{align}$$

#### II.2.2 用RM推导迭代式

1. 原目标问题：求$V^{\pi}(s)$
   - 符号： $V^{\pi}(s)= E_{a\sim\pi(s),s'\sim P(S'|s,a)}[r(s, a, s') + \gamma V^{\pi }(S')] $
   - 简记为：$V^{\pi}(s) = E[R(s)+\gamma V^{\pi}(S^{'})|s]$

2. 样本：t时刻的状态记为$s_t$，对应1个样本：$(s_t, a_t, s_{t+1}, r_{t+1})$

3. **分析**：
   - <font color=norange>原目标问题表达式右边的期望表达式中有$V^{\pi}(s')$，导致右式嵌套了左式，看起来和一般的RM算法求解案例不同。但实际上，在给定策略$\pi$和环境模型$R, P(S'|s,a)$之后，右边的期望值本质上是s的函数。由于期望的作用，该式中没有不确定性，对应一个scalar constant，就是$V^{\pi}(s)$的真实值。</font>
   - 唯一的差别在于，这里要求多个状态s对应的状态函数均值，而不是1个。

4. 转化为RM问题：$g(w(s))=w(s)-V^{\pi}(s)=w-E[R(s)+\gamma V^{\pi}(S^{'})|s]=0$
$$\begin{align}
g(w(s_t)) & = w(s_t) - E[R(s_t)+\gamma V^{\pi}(S_{t+1})|s_t] \\
\tilde g(w(s_t)) & = w(s_t) - [r_{t+1}+\gamma V^{\pi}(s_{t+1})] \\
\eta_t & = \tilde g(w(s_t)) - g(w(s_t)) \\
& = E[R(s_t)+\gamma V^{\pi}(S_{t+1})|s_t] - [r_{t+1}+\gamma V^{\pi}(s_{t+1})]
\end{align}$$

5. 迭代式：
   - 直接代入RM算法得到的迭代式：
$$\begin{align}
w_{t+1}(s_{t}) & = w_t(s_t) -\alpha_t(s_t)\tilde g(w_t(s_t)) \\
& = w_t(s_t) -\alpha_t(s_t)[w_t(s_t) - (r_{t+1}+\gamma V^{\pi}(s_{t+1}))]
\end{align}$$
     - 式中$V^{\pi}(s_{t+1})是状态s_{t+1}$的真实状态价值。如果该值已知，那么直接代入迭代过程，本质上就是在做supervised learning。这也是RM算法的迭代方式。可是该值在迭代过程中并不知道，它本身也是迭代求解的对象。
   - <font color=blue>**在当前步中对$V^{\pi}(s_{t+1})$的最佳估计量是$w_t(s_{t+1})$，将$V^{\pi}(s_{t+1})$替代为$w_t(s_{t+1})$就得到了basic TD算法的迭代式：**</font>
$$w_{t+1}(s_{t}) = w_t(s_t) -\alpha_t(s_t)[w_t(s_t) - (r_{t+1}+\gamma w_t(s_{t+1}))]
$$
       - 符号说明：t时刻状态记为$s_t$。给定策略$\pi$，t时刻对状态s的state value的估计记为$w_t(s)$

#### II.2.3 迭代式的收敛性：almost surely convergence

- <font color=brown> [证明详见math of RL 7.1.3节] </font>
- 简单说明：证明需要用到Dvorotzky's theorem的扩展形式
  - 迭代式转换为Dvorotzky's theorem对应的迭代式形式如下：
$$\begin{align}
w_{t+1}(s_{t}) 
& = w_t(s_t) -\alpha_t(s_t)[w_t(s_t) - (r_{t+1}+\gamma w_t(s_{t+1}))]\\
w_{t+1}(s_{t})  - w^{\pi }(s_{t})
& = w_t(s_t)  - w^{\pi }(s_{t})-\alpha_t(s_t)[w_t(s_t) - w(s_{t}) - (r_{t+1}+\gamma w_t(s_{t+1}) - w^{\pi }(s_{t}))]\\
\\
\Delta_{t+1} & = \Delta_t - \alpha_t[\Delta_t - (r_{t+1}+\gamma w_t(s_{t+1}) - w^{\pi }(s_{t}))] \\
& = (1-\alpha_t)\Delta_t+\alpha_t[r_{t+1}+\gamma w_t(s_{t+1}) - w^{\pi }(s_{t})] \\
& = (1-\alpha_t)\Delta_t+\alpha_t(\eta_t)\\
其中，\eta_t & =r_{t+1}+\gamma w_t(s_{t+1}) - w^{\pi }(s_{t})
\end{align}$$
      - <font color=red>$\eta_t中包含了对v_t(s)$而言来自外部的不确定性，此时可以证明它满足Dvorotzky's theorem所需性质，从而保证了迭代的收敛特征。</font>

### II.3 算法的性质和特征

#### II.3.1 迭代式分析

$$
\begin{align} 
\overset{新估计}{\overbrace{v^{\pi}_{t+1}(s_t)}} 
& = \underset{旧估计}{\underbrace{v^{\pi}_t(s_t)}} - \alpha_t\left [ \underset{TD\ error}{\underbrace{v^{\pi}_t(s_t)-(\overset{TD\ target\ value}{\overbrace{r_{t+1} + \gamma v^{\pi}_t(s_{t+1})} } )}}  \right ] \\
& = (1-\alpha_t)\underset{旧估计}{\underbrace{v^{\pi}_t(s_t)}}+\alpha_t (\underset{TD\ target\ value}{\underbrace{r_{t+1} + \gamma v^{\pi}_t(s_{t+1})} } )
\end{align}
$$

1. 新估计是旧估计和TD target value的加权平均。所谓target value的意思就是新的样本会让估计值从旧估计结果向这个value方向偏移，因此叫target value。
2. TD error也称为temperal-difference。它反映了时刻t对state value的估计值与新的单个样本获得的估计值的差异，也体现了新的单个样本提供的信息量大小。
3. 这个迭代的前提是给定策略$\pi$，不能用在策略不断变化的场景。

#### II.3.2 TD learning和MC learning的特征对比

| TD | MC |
|---------|---------|
| incremental，因此可以处理连续任务，可以处理实时learning任务   | non-incremental，MC exploring starter要用整个episode从后往前取样本点，因此不能处理连续实时任务   |
| bootstrapping，$v_{t+1}(s_t)依赖v_t(s_t)$。所以需要对估计量做initial guess，且initial guess的数值会影响算法效果 | non-bootstrapping,$v_{t+1}(s_t)不依赖v_t(s_t) $   |
| low variance(almost surely convergence), 但有bias   | unbias, 但high variance   |

## III. Tabular Sarsa：TD learning of action value

- 用RM迭代估计$Q^{\pi}(s, a)$，<font color=blue>做policy evaluation</font>。<font color=green>与basic TD的区别就是把对$V^{\pi}(s)$的估计改成直接估计$Q^{\pi}(s, a)$。</font>

### III.1 Sarsa algorithm

#### III.1.1 迭代式

$$
\begin{align} 
q_{t+1}(s_t, a_t) 
& \leftarrow q_t(s_t, a_t) -\alpha_t(s_t, a_t) [q_t(s_t, a_t) -(r(s_t,a_t,s_{t+1}) + \gamma q_t(s_{t+1}, a_{t+1})) ]\\
& = q_t(s_t, a_t) -\alpha_t(s_t, a_t) [q_t(s_t, a_t) -( r_{t+1} + \gamma q_t(s_{t+1}, a_{t+1}) ) ]\\
q_{t+1}(s, a)& \leftarrow q_t(s,a), \forall (s, a)\ne (s_t, a_t) 
\end{align}
$$

#### III.1.2 用RM算法推导迭代式

1. 目标问题：行为价值函数的Bellman Equation
$$q_{\pi }(s, a)= E[R+\gamma q^{\pi }(S', A')|(s, a)]$$

2. RM迭代式推导：
   - 转化问题：
$$\begin{align}
g(w(s, a)) & = w(s,a)-E[R+\gamma q^{\pi }(S', A')|(s, a)] \\
\tilde g(w(s, a)) & = w(s,a)-[r(s,a,s^{'})+\gamma q^{\pi }(s^{'}, a^{'})] \\
\eta(s,a) & = \tilde g(w(s, a)) - g(w(s, a)) \\
& = [r(s,a,s^{'})+\gamma q^{\pi }(s^{'}, a^{'})]-E[R+\gamma q^{\pi }(S', A')|(s, a)]\\
\end{align}$$
   - 迭代式：
$$\begin{align}
\tilde g(w(s, a)) & = w(s,a)-[r(s,a,s^{'})+\gamma q^{\pi }(s^{'}, a^{'})] \\
w_{t+1}(s_t,a_t) & = w_t(s_t,a_t)-\alpha_t(s_t,a_t)*\tilde g(w_t(s_t, a_t)) \\
& = w_t(s_t,a_t)-\alpha_t(s_t,a_t)*[w_t(s_t,a_t)-(r_{t+1}+\gamma w_t(s_{t+1}, a_{t+1})) ]
\end{align}$$

#### III.1.3 收敛性：almost surely convergence

- <font color=brown>证明思路和TD learning一样</font>

#### III.1.4 算法

- 算法伪码
   - <img src="./pics/sarsa.png" alt="alt text" width="560"/>

### III.2 n-step Sarsa algorithm

#### III.2.1 思路

- <font color=blue>取MC和Sarsa的折中，使用多几步的真实reward+下一个状态的价值估计，以此来控制bias-variance tradeoff。</font>

#### III.1.2 算法

1. 迭代方式
$$
\begin{align}
& Q^{\pi }_{t+1}(s_t,a_t) = \\
& Q^{\pi }_t(s_t,a_t)-\alpha_t \left [Q^{\pi }_t(s_t,a_t)-\left ( r_{t+1} + \gamma r_{t+2}+...+ \gamma^{n-1} r_{t+n} +\gamma^nQ^{\pi}_t(s_{t+n}, a_{t+n}) \right ) \right ]\\
\end{align}
$$
2. 详细算法（略）

## IV. Tabular Q-learning: TD learning of optimal action value

### IV.1 思路

1. 回顾Sarsa的算法思路：Sarsa在找最优策略的时候要分成两步，先做policy evaluation，再做policy improvement。原因是它的迭代式是在保持策略$\pi$不变的条件下估计action value$Q^{\pi}(s, a)$。得到action value的估计值之后再做policy improvement。
2. Q-learning的出发点：<font color=blue>**直接用RM算法的迭代方式找optimal action value。**</font>

### IV.2 迭代式

#### IV.2.1 迭代式

$$
\begin{align} 
sample\_q(s_t, a_t)
& = r(s_t,a_t,s_{t+1}) + \gamma \underset{a}{max}q^{\pi}_t(s_{t+1}, a)\\
& = r_{t+1} + \gamma \underset{a}{max}q^{\pi}_t(s_{t+1}, a)\\
q_{t+1}(s_t, a_t) 
& \leftarrow q_{t}(s_t, a_t)-\alpha_t \left [q_{t}(s_t, a_t)-sample\_v(s_t)\right ]\\
& = q_{t}(s_t, a_t)-\alpha_t \left [q_{t}(s_t, a_t)-\left ( r_{t+1} + \gamma \underset{a}{max}q^{\pi}_t(s_{t+1}, a)\right ) \right ]\\
q_{t+1}^{\pi}(s, a)& \leftarrow q^{\pi}_t(s, a), \forall (s, a)\ne (s_t, a_t)  \\
\end{align}
$$

- **和Sarsa的取样方式差异**: sarsa在每次迭代的时候要用$(s_t, a_t, r_{t+1}, s_{t+1}, a_{t+1})$，但Q-learning只用$(s_t, a_t, r_{t+1}, s_{t+1})$

#### IV.2.2 RM算法推导迭代式

1. 目标问题：行为价值函数的Bellman Optimality Equation
$$q(s, a)  = E[R+\gamma \underset{a^{'}}{max}q(S^{'}, a^{'})|(s, a)]$$

2. RM迭代式推导：
   - 转化问题：
$$\begin{align}
g(w(s, a)) & = w(s,a)- E[R+\gamma \underset{a}{max}q(S', a)|(s, a)]\\
\tilde g(w(s, a)) & = w(s,a)-[r(s,a,s^{'})+\gamma \underset{a^{'}}{max}q(s^{'}, a^{'})] \\
w_{t+1}(s_t,a_t) & = w_t(s_t,a_t)-\alpha_t(s_t,a_t)*\tilde g(w_t(s_t, a_t)) \\
& = w_t(s_t,a_t)-\alpha_t(s_t,a_t)*[w_t(s_t,a_t)-(r_{t+1}+\gamma \underset{a\in \mathcal{A_{t+1}}}{max}w_t(s_{t+1}, a)) ]
\end{align}$$

### IV.3 两种算法实现方式：off-policy 和 on-policy learning

#### IV.3.1 on-policy和off-policy的使用场景区别

- <font color=blue>**为什么Sarsa和MC learning要用on-policy learning，而Q-learning可以用off-policy learning？**</font>
  - 因为Q-leaning要求解的目标问题是BOE，而Sarsa和MC learning要求解的问题是Bellman Equation。
  - Bellman Equation评估的是给定策略$\pi$。
  - BOE并没有对应策略，可以直接选择每种state-action条件下的optimal value和optimal policies。
    <img src='pics/off-policy.png' width='100%'>

#### IV.3.2 Q-learning算法的on-policy实现(实践中不用)

1. on-policy learning
   - 此时生成数据的策略也使用被优化的那个策略。
   - 算法伪码：
     - <img src="./pics/qlearningon.png" alt="alt text" width="560"/>

#### IV.3.3 Q-learning算法的off-policy实现(实践中使用的方法)

- Q-learning通常是off-policy learning的形式
- 这时候专门有一个behavior策略$\pi_b$来生成数据，然后target策略会持续迭代优化。
- 算法伪码：
     - <img src="./pics/qlearningoff.png" alt="alt text" width="560"/>

## IV. 小结：用统一的视角来看待MC/Sarsa/Q-learning

- 目标问题：
  - <img src="./pics/target_expression.png">

| algs | 求解目标 |
|---------|---------|
| Sarsa   | $E[R_{t+1}+\gamma Q(s_{t+1}, a_{t+1})|s_t, a_t]$   |
| n-step Sarsa   | $E[R_{t+1}+\gamma R_{t+2}+...+\gamma^n Q(s_{t+n}, a_{t+n})|s_t, a_t]$   |
| Q-learning   | $E[R_{t+1}+\gamma max_a Q(s_{t+1}, a)|s_t, a_t]$  |
| MC   | $E[R_{t+1}+\gamma R_{t+2}+\gamma^2R_{t+3}+...]$   |


- 迭代式形式：
  - <img src="./pics/iterate_expression.png">

| algs | 迭代形式 |
|---------|---------|
| Sarsa   | $r_{t+1}+\gamma q_t(s_{t+1}, a_{t+1})$   |
| n-step Sarsa   |$r_{t+1}+\gamma r_{t+2}+...+\gamma^n q_t(s_{t+n}, a_{t+n})$  |
| Q-learning   | $r_{t+1}+\gamma max_aq_t(s_{t+1}, a)$ |
| MC   | $r_{t+1}+\gamma r_{t+2}+\gamma^2r_{t+3}+...$   |